### LTE/PEP SD-GC Pilot Issuance through GCOS

We will need to follow the steps below in order to ensure that the Quintrace (QT) platform can interface properly with the GranularCertOS (GCOS) registry for the purposes of time-shifting GC attributes.

1) Create a new (*storage validator*) user on GCOS with the organisation listed as `Quintrace`. The endpoint created for converting between the QT and GCOS storage allocation records format will check for this to make sure that no other user can access the method.

2) Create a new (*trading*) user for the LTE escrow account acting on behalf of Google 
	1) *Should each party LTE is acting on behalf of have its own account? I think so, because otherwise a separate ledger would be needed to keep track of ownership*.
3) Create new accounts for the QT and LTE users. These accounts must be whitelisted in both directions with each other to enable transfer of (SD)GCs.
4) Create a new device for Byrd Ranch and assign to the QT account. QT will need the ID of this device when submitting their allocation records.
	1) *Alternatively, we can accept the name as in the file and map to the device ID internally through searching by name*.
5) Import the GCs from a CSV provided by QT/LTE/Flexidao through the endpoint `/certificate/import` into the LTE escrow account for Google.
	1) *For the pilot, we will not add functionality to check that the imported certificates are either cancelled or marked as exported directly. We will need to rely on some form of evidence external to the platform that this has been performed*
6) The LTE escrow account for Google transfers the certificates it has imported to the QT battery operator account.
7) QT queries its account to verify that the GCs it wishes to time-shift are present.
8) QT 'generates' the charge/discharge allocation record.
	1) *In reality this is already done, but in practice the linking of GCs to each allocation record should happen after QT is aware of which is has available to allocate.*
9) QT submits the asset position table to the endpoint `quintrace/submit_storage_records` so that GCOS can create the SCR/SDR entities.
	1) *This step feels necessary as we can't actually derive the charge record from the submitted discharge table, as such we cannot calculate or validate the loss factor.*
	2) *Let's catch up on this can see whether the charge records in the QT methodology PDF can be submitted if position tables are too difficult in every case.*
10) GCOS converts this table into immutable `StorageRecord` entities, representing the SCR/SDR records as defined in the EnergyTag Standard, and submits confirmation to QT.
11) QT submits the discharge record table to GCOS through the endpoint `quintrace/submit_allocation_records`.
	1) *I am minded to provide an option at this stage for QT to specify whether to cancel the GCs allocated in their table if not already, or to be strict in the validation and say they need to be cancelled ahead of the allocation/SDGC-issue request.*
	2) *Advantage of the latter approach is that they can perform this whole workflow in two rather than three operations - the downside is less visibility on the actions being performed, potentially leading to unintentional cancelations if there are mistakes in the allocation table.*
12)  GCOS uses the to SCRs/SDRs from the previous step to convert this table to mutable `AllocatedStorageRecord` entities, containing information on how each charge and discharge event has been allocated and the requisite GCs that need to be cancelled.
	1) *To avoid the need for QT to know the IDs of the SCR/SDR pairs ahead of submission, we will match on the `hour`/`charge_hour` fields instead, assuming that QT have correctly validated their submission.*
13) GCOS issues SDGCs using these records and notifies QT on completion. 
14) QT transfers these SDGCs to the LTE escrow account for Google.
	1) *At this stage, we can put up an `export` function to replicate the process of sending the SDGCs back to Flexidao, by marking their status as `EXPORTED` and generating a CSV matching the format of the imported GCs.*


**Manual Issuance from Example Records**

SDR Volume = 5,257,248
GC to cancel = 5,210,000
SDGC Volume = 5,004,300 (i.e. discharge volume from charge record - remaining energy)

Therefore:
SDGC (5,004,300) = SDR Volume (5,257,248) * SDR Proportion, => sdr_prop = 0.95188585
GC Cancel (5,210,000) = SDR Volume (5,257,248) * SDR Proportion (0.95188585) / efficiency
=> Efficiency Factor = 0.96051823

In [2]:
import pandas as pd
import datetime
import httpx
import logging.config

from gc_registry.account.schemas import AccountBase, AccountWhitelist
from gc_registry.user.schemas import UserCreate, UserRoles
from gc_registry.certificate.schemas import GranularCertificateQuery, GranularCertificateTransfer, GranularCertificateCancel
from gc_registry.device.models import DeviceBase
from gc_registry.core.models.base import DeviceTechnologyType, EnergySourceType

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#### Set up client connection and tokens

In [17]:
client = httpx.Client(base_url="http://localhost:8000")
csrf_token = client.get("/csrf-token").json()["csrf_token"]
client.headers["X-CSRF-Token"] = csrf_token

LOGGING_CONFIG = {
    "version": 1,
    "handlers": {
        "default": {
            "class": "logging.StreamHandler",
            "formatter": "http",
            "stream": "ext://sys.stderr"
        }
    },
    "formatters": {
        "http": {
            "format": "%(levelname)s [%(asctime)s] %(name)s - %(message)s",
            "datefmt": "%Y-%m-%d %H:%M:%S",
        }
    },
    'loggers': {}
}

logging.config.dictConfig(LOGGING_CONFIG)

token = client.post("/auth/login", data={"username": "admin_user@usermail.com", "password": "admin"}).json()

headers = {"Authorization": f"Bearer {token["access_token"]}", "X-CSRF-Token": csrf_token}

token

#### Create new users for LTE and PEP (Quintrace)

If this notebook is to be run more than once, this step will return a failure message if the accounts already exist.

In [ ]:
quintrace_user = UserCreate(
    name="Quintrace User",
    email="quintrace_user@private-energypartners.com",
    password="quintrace",
    role=UserRoles.STORAGE_VALIDATOR,
    organisation="Quintrace"
)
lte_user = UserCreate(
    name="LTE User",
    email="lte_user@leveltenenergy.com",
    password="leveltenenergy",
    role=UserRoles.TRADING_USER,
    organisation="LevelTen Energy"
)

quintrace_response = client.post("user/create", data=quintrace_user.model_dump_json(), headers=headers)
lte_response = client.post("user/create", data=lte_user.model_dump_json(), headers=headers)

print(f"Quintrace user created: {quintrace_response.json()}")
print(f"LTE user created: {lte_response.json()}")

In [ ]:
# Get the user IDs for the LTE and Quintrace users for downstream operations
quintrace_user_id = client.get("user/get_user_by_email", params={"email": "quintrace_user@private-energypartners.com"}, headers=headers).json()["id"]
lte_user_id = client.get("user/get_user_by_email", params={"email": "lte_user@leveltenenergy.com"}, headers=headers).json()["id"]

print(f"Quintrace user ID: {quintrace_user_id}")
print(f"LTE user ID: {lte_user_id}")

In [ ]:
# Log in as the two new user accounts for device creation and whitelisting purposes
lte_token = client.post("/auth/login", data={"username": "lte_user@leveltenenergy.com", "password": "leveltenenergy"}).json()["access_token"]
quintrace_token = client.post("/auth/login", data={"username": "quintrace_user@private-energypartners.com", "password": "quintrace"}).json()["access_token"]

lte_headers = {"Authorization": f"Bearer {lte_token}"}
quintrace_headers = {"Authorization": f"Bearer {quintrace_token}"}

#### Create accounts and storage device

For the QT user, create an account and a storage device for Byrd Ranch. For the LTE users, create a storage account to manage the GCs imported on behalf of Google.

In [ ]:
quintrace_account = AccountBase(account_name="Quintrace Account", user_ids=[quintrace_user_id])

quintrace_account_result = client.post("account/create", data=quintrace_account.model_dump_json(), headers=headers)
quintrace_account_result.json()

In [ ]:
lte_account = AccountBase(account_name="LTE Account", user_ids=[lte_user_id])

lte_account_result = client.post("account/create", data=lte_account.model_dump_json(), headers=headers)
lte_account_result.json()

In [ ]:
# Whitelist the two newly created accounts in both directions
lte_whitelist = AccountWhitelist(add_to_whitelist=[quintrace_account_result["id"]])

whitelist_result = client.post(
    f"account/update_whitelist/{lte_account_result['id']}", 
    data=lte_whitelist.model_dump_json(), 
    headers=lte_headers)

print(f"LTE whitelist: {whitelist_result.json()}")

quintrace_whitelist = AccountWhitelist(add_to_whitelist=[lte_account_result["id"]])

whitelist_result = client.post(
    f"account/update_whitelist/{quintrace_account_result['id']}", 
    data=quintrace_whitelist.model_dump_json(), 
    headers=quintrace_headers)

print(f"Quintrace whitelist: {whitelist_result.json()}")

In [ ]:
byrd_ranch = DeviceBase(
    account_id=quintrace_account_result["id"],
    device_name="Byrd Ranch",
    technology_type=DeviceTechnologyType.battery_storage,
    grid="ERCOT",
    energy_source_type=EnergySourceType.battery_storage,
    operational_date=datetime.datetime(2023, 7, 1),
    capacity=50,
    peak_demand=50,
    location=(29.056, -95.687),
    is_storage=True
)

response = client.post("device/create", data=byrd_ranch.model_dump_json(), headers=quintrace_headers)
print(f"Byrd Ranch device created: {response.json()}")

### Import GCs from Flexidao

We will use the import endpoint to accept a CSV containing the GCs that we are importing from Flexidao. To be compatible with the information provided by QT, we have assumed the values of fields not in the certificate inititally shared that we can correct down the line if needed. These GCs are imported into the LTE account to represent LTE acting on behalf of Google in retrieving them from Flexidao.

In [ ]:
gcos_import_df = pd.read_csv("data/gc_pilot_certs_gcos_format.csv")

response = client.post(
    "certificate/import",
    headers={"Authorization": f"Bearer {lte_token}"},
    data={"account_id": lte_account.id},
    files={"file": ("gc_pilot_certs_gcos_format.csv", gcos_import_df.to_csv(index=False))},
)

print(response.json())

Check that the imported GC bundles are in the LTE account. This step can be performed by LTE directly.

In [ ]:
gc_bundle_query = GranularCertificateQuery(
    user_id=lte_user["id"],
    source_id=lte_account.id,
)

gc_bundles_from_query = client.post("certificate/query", data=gc_bundle_query.model_dump_json(), headers=lte_headers)

pd.DataFrame(gc_bundles_from_query.json()["granular_certificate_bundles"]).sort_values("production_starting_interval")

LTE transfers the GCs into the QT storage validator account.

In [ ]:
transfer_action = GranularCertificateTransfer(
    user_id=lte_user["id"],
    source_id=lte_account.id,
    target_id=quintrace_account.id,
    granular_certificate_bundle_ids = gc_bundles_from_query.json()["granular_certificate_bundles"][0]["id"],
)

response = client.post("certificate/transfer", data=transfer_action.model_dump_json(), headers=headers)

response.json()

QT then verify that it received the correct GCs (QT can perform this step directly as they create their discharge records).

In [ ]:
gc_bundle_query = GranularCertificateQuery(
    user_id=quintrace_user["id"],
    source_id=quintrace_account.id,
)

gc_bundles_from_query = client.post("certificate/query", data=gc_bundle_query.model_dump_json(), headers=headers)

pd.DataFrame(gc_bundles_from_query.json()["granular_certificate_bundles"]).sort_values("production_starting_interval")